In [14]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Connect to database via CRUD Module
shelter = AnimalShelter()  # Configuration is handled in CRUD implementation

# Class read method must support return of list object and accept projection json input
# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
if not df.empty:
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True

# FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Header
    html.Div([
        html.A(
            html.Img(src=f'data:image/png;base64,{encoded_image.decode()}', style={'height': '100px'}),
            href='https://www.snhu.edu',
            target='_blank'
        ),
        html.H1('Austin Animal Outcomes SNHU CS-340 Dashboard', style={'textAlign': 'center'}),
    ], style={'textAlign': 'center'}),

    html.Hr(),
    # Filters
    html.Div([
        dcc.RadioItems(
            id='filter-type',  # Ensure this ID matches the callback Input
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ], style={'textAlign': 'center'}),

    html.Hr(),
    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={"textAlign": "left"},
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
    ),
    html.Br(),
    html.Hr(),
    # Graph and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ]),

    # Footer
    html.Div(
        html.P("Matthew Bates is a student at SNHU", style={'textAlign': 'center', 'fontSize': '16px'}),
        style={'padding': '10px', 'backgroundColor': '#f1f1f1'}
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]  # Corrected ID from 'radio-items' to 'filter-type'
)
def update_dashboard(filter_type):
    if filter_type == 'water':
        query = {'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex': 'Intact Female', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
    elif filter_type == 'mountain':
        query = {'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 'sex': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
    elif filter_type == 'disaster':
        query = {'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 'sex': 'Intact Male', 'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
    else:
        query = {}
        
    filtered_data = shelter.read(query)
    df = pd.DataFrame.from_records(filtered_data)
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    breed_counts = dff['breed'].value_counts()
    
    top_breeds = breed_counts.nlargest(6)
    other_breeds = breed_counts.iloc[6:].sum()
    
    breeds_to_display = top_breeds.append(pd.Series([other_breeds], index=['Other Breeds']))
    
    fig = px.pie(
        names=breeds_to_display.index,
        values=breeds_to_display.values,
        title="Top Breeds"
    )
    
    return [dcc.Graph(figure=fig)]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0  # Get the first selected row
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},  # Sets the size of the map
              center=[30.75, -97.48], zoom=10, children=[  # Initial center and zoom of map
                  dl.TileLayer(id="base-layer-id"),
                  dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],  # Position of marker
                           children=[
                               dl.Tooltip(dff.iloc[row, 4]),
                               dl.Popup([
                                   html.H1("Animal Name"),  # Header
                                   html.P(dff.iloc[row, 9])  # Content
                               ])
                           ])
              ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:23236/


/tmp/ipykernel_1840318/3272008918.py:150: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

